In [2]:
import pandas as pd
import requests
import json

# Main function to retrieve data from Rapid API (AeroDataBox) and send it to SQL
def retrieve_and_send_data():
  connection_string = create_connection_string()
  cities_df = fetch_cities_data(connection_string)
  cities_airports_df = icao_airport_codes(cities_df)
  airports_df = icao_airport_names(cities_df)
  store_cities_airports(cities_airports_df, connection_string)
  store_airports(airports_df, connection_string)
  return "Data has been updated"

# Function to create a connection string to save data in SQL database
def create_connection_string():
  schema = "YOUR_SQL_DATABASE_NAME"
  host = "11.11.111.111"
  user = "root"
  password = "YOUR_PASSWORD"
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# Function to get cities data (city name,id and lat/long) from SQL
def fetch_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)

# Function to get icao data (airport code) from API
def icao_airport_codes(cities_df):
    cities_airports=[]
    for _, city in cities_df.iterrows():
        city_id = city['city_id']
        latitude = city['latitude']
        longitude = city['longitude']
        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitude}/{longitude}/km/50/10"
        querystring = {"withFlightInfoOnly":"true"}
        headers = {
                	"x-rapidapi-key": "YOUR_API_KEY",
                	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }
        response = requests.get(url, headers=headers, params=querystring)
        airports_json = response.json()
        for i in airports_json['items']:
            airport_icao = {
                        "city_id": city_id,
                        "airport_icao": i.get("icao",None)
            }
            cities_airports.append(airport_icao)
    cities_airports_df = pd.DataFrame(cities_airports)
    return cities_airports_df

# Function to get airport name and icao data (airport code) from API
def icao_airport_names(cities_df):
    airports=[]
    for _, city in cities_df.iterrows():
        city_id = city['city_id']
        latitude = city['latitude']
        longitude = city['longitude']
        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitude}/{longitude}/km/50/10"
        querystring = {"withFlightInfoOnly":"true"}
        headers = {
                	"x-rapidapi-key": "YOUR_API_KEY",
                	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }
        response = requests.get(url, headers=headers, params=querystring)
        airports_json = response.json()
        for i in airports_json['items']:
            airport_name = {
                        "airport_icao": i.get("icao",None),
                        "airport_name": i.get("name",None)
            }
            airports.append(airport_name)

    airports_df = pd.DataFrame(airports)
    return airports_df
    
#Function to store airport icao data as an SQL database
def store_cities_airports(cities_airports_df, connection_string):
  cities_airports_df.to_sql('cities_airports',
                    if_exists='append',
                    con=connection_string,
                    index=False)

#Function to store airport name data as an SQL database
def store_airports(airports_df, connection_string):
  airports_df.to_sql('airports',
                    if_exists='append',
                    con=connection_string,
                    index=False)    

In [4]:
retrieve_and_send_data()

'Data has been updated'